In [1]:
from __future__ import print_function # so print doesn't show brackets
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
import pickle
pickle.HIGHEST_PROTOCOL = 2
sys.path.append(os.path.join("..", "Libraries","QML_lib"))
import Evo as evo
import DataBase 
from QMD import QMD #  class moved to QMD in Library
import QML
import ModelGeneration 
import BayesF
import matplotlib.pyplot as plt
from pympler import asizeof
import matplotlib.pyplot as plt
paulis = ['x', 'y', 'z'] # will be chosen at random. or uncomment below and comment within loop to hard-set

import time as time 
import argparse
parser = argparse.ArgumentParser(description='Pass variables for (I)QLE.')
import redis
redis_db = redis.StrictRedis(host="localhost", port=6379, db=0)
learned_models_info = redis.StrictRedis(host="localhost", port=6379, db=1)


num_particles=50
num_experiments = 30
qle=True

initial_op_list = ['x', 'y', 'z']
true_op = random.choice(initial_op_list)
global_true_op = true_op

op = DataBase.operator(global_true_op)
n_pars = op.num_constituents
true_params = [np.random.rand() for i in range(n_pars)]

qmd = QMD(
    initial_op_list=initial_op_list, 
    true_operator=true_op, 
    true_param_list=true_params, 
    num_particles=num_particles,
    num_experiments = num_experiments, 
    qle=qle,
    max_num_branches = 0,
    max_num_qubits = 2, 
    parallel = True
)

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)



Running  QLE  for true operator  x  with parameters :  [0.2730166517163882]
probe dict given to add model function
Model  x  not previously considered -- adding.
Initialization Ready
probe dict given to add model function
Model  y  not previously considered -- adding.
Initialization Ready
probe dict given to add model function
Model  z  not previously considered -- adding.
Initialization Ready


In [2]:
qmd.learnModel(model_name = true_op, use_rq=True)

Model x added to queue.


In [ ]:
a=qmd.pullField(true_op, field='Reduced_Model_Class_Instance')

In [ ]:
a.FinalParams

In [ ]:
import redis
redis_db = redis.StrictRedis(host="localhost", port=6379, db=0)
learned_models_info = redis.StrictRedis(host="localhost", port=6379, db=1)


In [11]:
a = time.time()
for i in range(10):
    pickle.loads(learned_models_info['1.0'])
b = time.time()
print("Time = ", (b-a)/10)    

Time =  0.00101039409637


In [5]:

pickle.loads(learned_models_info['QMDInfo'])

{'debug_directory': None,
 'num_experiments': 30,
 'num_particles': 50,
 'num_probes': 20,
 'pgh_prefactor': 1.0,
 'probe_dict': {(0,
   0): array([ 0.79689323+0.24145942j,  0.30643444+0.46125531j]),
  (0, 1): array([ 0.79689323+0.24145942j,  0.30643444+0.46125531j]),
  (0, 2): array([ 0.01535157+0.31402808j,  0.63338383+0.43967839j,
         -0.12509525+0.16754495j,  0.13864971+0.49367464j]),
  (1, 0): array([ 0.53462877+0.20344984j,  0.77937869+0.2556347j ]),
  (1, 1): array([ 0.53462877+0.20344984j,  0.77937869+0.2556347j ]),
  (1, 2): array([ 0.21942296+0.16624296j,  0.14503397+0.48000182j,
          0.32540996+0.22343305j,  0.23985440+0.67781719j]),
  (2, 0): array([ 0.02704372+0.82158708j,  0.24542669+0.51383757j]),
  (2, 1): array([ 0.02704372+0.82158708j,  0.24542669+0.51383757j]),
  (2, 2): array([-0.22993935+0.36977411j, -0.44044007+0.54050369j,
         -0.04317551+0.2985313j , -0.12931745+0.46535497j]),
  (3, 0): array([ 0.19766997+0.41050683j,  0.71815461+0.52598925j]),
  

In [ ]:
from pympler import asizeof

asizeof.asizeof(learned_models_info['1.0'])

In [ ]:
myfile=open('../Libraries/QML_lib/test_pickled_model_dict.pkl', 'rb')
a=pickle.load(myfile)
myfile.close()

In [ ]:
a

In [4]:
import rq 
from redis import Redis
red_con = Redis()

rq.registry.FinishedJobRegistry('default', connection=red_con).get_job_ids()

[u'f3c2b374-f269-4c57-96e0-1541e25ee3c4',
 u'a0a2852f-b375-48ed-99c4-7be0a15e5a70']

In [ ]:
red_con.keys()

In [ ]:
red_con.keys()

In [ ]:
import json

In [ ]:
json.dumps